In [ ]:
import torch
import io

from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.utils import ngrams_iterator
from torchtext.data.utils import get_tokenizer
from torchtext.utils import unicode_csv_reader

In [ ]:
from ir_classification import datasets, models
from ir_classification import vocab as ir_vocab
from ir_classification import train

In [ ]:
vocab = ir_vocab.create_vocab_from_tsv("../datasets/systematic_review/phase1.train.shuf.tsv", [2], ngrams=2)
data_columns = [2]
dataset = datasets.TSVRawTextMapDataset("../datasets/systematic_review/phase1.train.shuf.tsv", data_columns)
label_transform = lambda x: x if x > 0 else 0
tokenizer = get_tokenizer("basic_english")
text_transform = lambda x: list(ngrams_iterator(tokenizer(x), 2))
dataloader = datasets.create_torch_dataloader(dataset, vocab,  label_transform, text_transform, weighted=True)

In [ ]:
num_classes = 2
vocab_size = len(vocab)
embedding_size = 64
model = models.EmbeddingBagLinearModel(vocab_size, embedding_size, num_classes)

In [ ]:
import time
EPOCHS = 10 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)

train.train_epoch(0, model, optimizer, criterion, dataloader)